<a href="https://colab.research.google.com/github/karube044/100/blob/master/9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# 80. ID番号への変換
# 準備
# 問50で作成した学習、検証、評価データを用いる
import pandas as pd
train = pd.read_csv('train.txt', header=0, sep='\t')
valid = pd.read_csv('valid.txt', header=0, sep='\t')
test = pd.read_csv('test.txt', header=0, sep='\t')

In [63]:
# 単語のID番号を付与する
import collections
import string

# 本文から記号を消去し、単語ごとに分けすべてリストに格納する
table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
l = []
for text in train['TITLE']:
  words = text.translate(table).split()
  for word in words:
    l.append(word)

# collections.Counterでリスト内の単語の出現頻度を辞書型として取得
c = collections.Counter(l)
# sorted()で辞書の要素を出現頻度の降順にリストとして取得
f = sorted(c.items(), key=lambda x: x[1], reverse=True)

# リストの順番に順位(単語ID)をつけていく(2回以上出現した単語のみ)
word_id={}
for i, (w, cnt) in enumerate(f):
  if cnt > 1:
    word_id[w]= i+1 


In [62]:
# 確認(リストに変換し、先頭から10個目を表示する)
tmp = list(word_id.items())
print(tmp[:10])
print(len(set(word_id.values())))

[('to', 1), ('s', 2), ('in', 3), ('UPDATE', 4), ('on', 5), ('as', 6), ('US', 7), ('for', 8), ('of', 9), ('The', 10)]
9397


In [14]:
# 与えられた単語列に対し単語IDを返す関数
def return_id(text):
  # 単語列の単語IDを入れるリスト
  ids = []
  # 同じように文から単語を取得し、それに対応する単語IDをリストに与える
  table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
  words = text.translate(table).split()
  for word in words:
    # get()で単語IDが与えられていない単語には0を返すようにする
    ids.append(word_id.get(word, 0))

  return ids

# 確認(適当な記事見出しを選択)
text = train['TITLE'][12]
print(text)
print(return_id(text))

UPDATE 2-Dollar General CEO to retire, Icahn's proposed merger in doubt
[4, 13, 55, 880, 60, 1, 6594, 2098, 2, 1568, 566, 3, 0]


In [15]:
# 81. RNNによる予測
# 標準的に必要となるライブラリ
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

In [16]:
# バッチ処理を行えるようにデータセットを作成する
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class MyDataset(Dataset):
  def __init__(self, xdata, ydata, return_id):
    self.data = xdata
    self.label = ydata
    self.return_id = return_id

  # len(Dataset)で返す値を指定
  def __len__(self):  
    return len(self.label)

  # Dataset[idx]で返す値を指定
  def __getitem__(self, idx):
    # 前問で作成した関数を用いて単語列をIDに変換し返す
    text = self.data[idx]
    x = self.return_id(text)
    y = self.label[idx]
    return x,y

# 文の長さが一定ではないのでPaddingを行ってDataLoaderの出力とする
def my_collate_fn(batch):
  # バッチサイズ分のデータを取得
  xdata, ydata = list(zip(*batch))
  xs = list(xdata)
  # Paddingを行う
  xs1 = []
  for k in range(len(xs)):
    ids = xs[k]
    xs1.append(torch.LongTensor(ids))
  # Paddingの数値は単語ID辞書に登録されている単語数+1とする
  xs1 = pad_sequence(xs1, batch_first=True, padding_value=len(word_id)+1)
  
  ys = list(ydata)
  ys1 = torch.LongTensor(ys)

  return xs1, ys1

In [17]:
# 記事のカテゴリ名をラベルに変更するための辞書(8章と同じもの)
category_dict = {'b': 0, 't': 1, 'e':2, 'm':3}
Y_train = torch.tensor(list(map(lambda x: category_dict[x], train['CATEGORY'])))
Y_valid = torch.tensor(list(map(lambda x: category_dict[x], valid['CATEGORY'])))
Y_test = torch.tensor(list(map(lambda x: category_dict[x], test['CATEGORY'])))

# それぞれのデータセットを作成
dataset_train = MyDataset(train['TITLE'], Y_train, return_id)
dataset_valid = MyDataset(valid['TITLE'], Y_valid, return_id)
dataset_test = MyDataset(test['TITLE'], Y_test, return_id)

In [18]:
# 確認(バッチ処理とするのでバッチサイズはデータ全て)
dataloader_train = DataLoader(dataset_train, batch_size=len(dataset_train), shuffle=True,
                        collate_fn=my_collate_fn)
d1 = dataloader_train.__iter__()
xs, ys = d1.next()
print(xs)
print(len(xs))
print(ys)

tensor([[1569,   15,  945,  ..., 9398, 9398, 9398],
        [ 155,    0, 3349,  ..., 9398, 9398, 9398],
        [1386,   10, 6794,  ..., 9398, 9398, 9398],
        ...,
        [1257, 1985, 2747,  ..., 9398, 9398, 9398],
        [3116,    2,  116,  ..., 9398, 9398, 9398],
        [ 424,  601, 2778,  ..., 9398, 9398, 9398]])
10672
tensor([1, 3, 2,  ..., 2, 0, 2])


In [19]:
# モデルの設定
class Nlp81(nn.Module):
  def __init__(self, vocab_size, embd_size, output_size, hid_size, padding_idx):
    super(Nlp81, self).__init__()

    # 埋め込み層でone-hotから単語ベクトルに変換
    self.embd = nn.Embedding(vocab_size, embd_size, padding_idx=padding_idx)

    # RNN層を定義
    # 活性化関数はtanhとする   
    self.rnn = nn.RNN(embd_size, hid_size, nonlinearity='tanh', batch_first=True)

    # 隠れ層を定義
    self.l1=nn.Linear(hid_size, output_size)
    # 今回は重みをランダムな値で(正規分布により)初期化する
    nn.init.normal_(self.l1.weight, 0.0, 1.0) 
        
  # 順方向の計算
  def forward(self,x):
    # 埋め込み層で単語ベクトルに直し
    ex = self.embd(x)
    # RNNユニットで計算する
    out, h_T = self.rnn(ex)

    # RNNでは各単語に対する出力と、最後の隠れ状態ベクトルが得られる
    # 最後の隠れ状態ベクトルは単語列の最後の単語に対する出力と等しいのでそちらを使用する
    # ソフトマックス関数を適用し求める(dim=-1で行のsoftmaxを指定)
    h1 = torch.softmax(self.l1(out[:,-1,:]), dim=-1)
    return h1

In [20]:
# 確認
# モデルの引数は単語の総数、単語ベクトル、出力ラベル、隠れ層の次元、Paddingの数値
# 単語の総数は単語ID辞書に登録されている単語数＋2(登録されていない単語+Padding)
model = Nlp81(len(word_id)+2, 300, 4, 50, len(word_id)+1)
for xs, ys in dataloader_train:
  y1_hat=model(xs)
  print(y1_hat)

tensor([[0.0287, 0.2458, 0.4089, 0.3166],
        [0.0286, 0.2487, 0.4042, 0.3185],
        [0.0264, 0.2724, 0.4065, 0.2947],
        ...,
        [0.0283, 0.2489, 0.4030, 0.3198],
        [0.0283, 0.2462, 0.4077, 0.3178],
        [0.0285, 0.2457, 0.4122, 0.3137]], grad_fn=<SoftmaxBackward>)


In [21]:
# 82. 確率的勾配降下法による学習

# 正解率と損失の計算のための関数
def cal_loss_accuracy(model, criterion, dataset):
  # 正解率、損失を求めるときには比較のためshuffleはFalseとする
  # バッチ処理で求める
  dataloader = DataLoader(dataset, batch_size=len(dataset), shuffle=False,
                        collate_fn=my_collate_fn)
  model.eval()
  with torch.no_grad():
    for xs, ys in dataloader:
      # 順方向による出力
      output = model(xs)
      # 損失の計算
      loss = criterion(output, ys).item()
      # 正解率の計算
      ans = torch.argmax(output,1)
      accuracy = ((ys == ans).sum().float() / len(ans)).item()
  return loss, accuracy

In [22]:
# モデルのインスタンス、損失関数、最適化関数の設定
model = Nlp81(len(word_id)+2, 300, 4, 50, len(word_id)+1)

# ラベルはPaddingを行っていないのでクロスエントロピーで気にする必要はない
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# データをロードする
dataloader_train = DataLoader(dataset_train, batch_size=len(dataset_train), shuffle=True,
                        collate_fn=my_collate_fn)

# 学習の開始
# 10エポックで終了とする
for ep in range(10):
  
  model.train()
  # データも81で取得したdataloader_trainを使用する
  for xs, ys in dataloader_train:
    output = model(xs) 
    loss = criterion(output, ys) 
    optimizer.zero_grad()
    loss.backward() 
    optimizer.step()

  loss_train, accuracy_train = cal_loss_accuracy(model, criterion, dataset_train)
  loss_test, accuracy_test = cal_loss_accuracy(model, criterion, dataset_test) 
  print('エポック数', ep)
  print('[訓練データ]損失：',loss_train,'正解率:',accuracy_train)
  print('[評価データ]損失：',loss_test,'正解率:',accuracy_test)


エポック数 0
[訓練データ]損失： 1.2951695919036865 正解率: 0.4222263991832733
[評価データ]損失： 1.294743537902832 正解率: 0.4220389723777771
エポック数 1
[訓練データ]損失： 1.283414602279663 正解率: 0.42306971549987793
[評価データ]損失： 1.2827931642532349 正解率: 0.4220389723777771
エポック数 2
[訓練データ]損失： 1.2742602825164795 正解率: 0.4253185987472534
[評価データ]損失： 1.2733274698257446 正解率: 0.4220389723777771
エポック数 3
[訓練データ]損失： 1.2694734334945679 正解率: 0.4300037622451782
[評価データ]損失： 1.2684270143508911 正解率: 0.4220389723777771
エポック数 4
[訓練データ]損失： 1.2669267654418945 正解率: 0.4315029978752136
[評価データ]損失： 1.2659424543380737 正解率: 0.4220389723777771
エポック数 5
[訓練データ]損失： 1.2652689218521118 正解率: 0.4320652186870575
[評価データ]損失： 1.2643778324127197 正解率: 0.4220389723777771
エポック数 6
[訓練データ]損失： 1.264090657234192 正解率: 0.43234631419181824
[評価データ]損失： 1.2632941007614136 正解率: 0.4220389723777771
エポック数 7
[訓練データ]損失： 1.2632015943527222 正解率: 0.43318966031074524
[評価データ]損失： 1.2625010013580322 正解率: 0.4220389723777771
エポック数 8
[訓練データ]損失： 1.2624998092651367 正解率: 0.4330959618091583
[評価データ]損失：

In [23]:
# 83. ミニバッチ化・GPU上での学習
# GPUに送るためのコード
device = torch.device("cuda:0" 
                     if torch.cuda.is_available()
                     else "cpu") 

In [24]:
# バッチサイズは、データロードのときにサイズを指定すれば変更できる
dataloader_train = DataLoader(dataset_train, batch_size=4, shuffle=True,
                        collate_fn=my_collate_fn)

# 確認
d1 = dataloader_train.__iter__()
xs, ys = d1.next()
print(xs)
print(len(xs))
print(ys)

tensor([[ 127,   17,  168,  978,   52,    2,   84,  113,   25,   20, 8183, 9398],
        [  51,  402,    5, 4759,   88,  484,    6,   67, 3971, 5246,    8,  830],
        [  22,   19,   34,   30,  587, 1655,    5,   14, 1591, 9398, 9398, 9398],
        [ 404,   60,   36,  760,  306,  476,  108,  217, 7129, 1347,    3,  481]])
4
tensor([2, 0, 2, 1])


In [25]:
# モデルの設定(81の流用(重みなどの初期化は省いている))
class Nlp83(nn.Module):
  def __init__(self, vocab_size, embd_size, output_size, hid_size, padding_idx):
    super(Nlp83, self).__init__()

    self.embd = nn.Embedding(vocab_size, embd_size, padding_idx=padding_idx)
    self.rnn = nn.RNN(embd_size, hid_size, nonlinearity='tanh', batch_first=True)
    self.l1=nn.Linear(hid_size, output_size)
        
  def forward(self,x):
    ex = self.embd(x)
    out, h_T = self.rnn(ex)
    h1 = torch.softmax(self.l1(out[:,-1,:]), dim=-1)
    return h1

In [26]:
# 正解率と損失の計算のための関数(GPU用)
def cal_loss_accuracy_gpu(model, criterion, dataset):
  dataloader = DataLoader(dataset, batch_size=len(dataset), shuffle=False,
                        collate_fn=my_collate_fn)
  model.eval()
  with torch.no_grad():
    for xs, ys in dataloader:
      # 計算を行うためのベクトルをGPUに移動する
      xs = xs.to(device)
      ys = ys.to(device)
      output = model(xs)
      loss = criterion(output, ys).item()
      ans = torch.argmax(output,1)
      accuracy = ((ys == ans).sum().float() / len(ans)).item()
  return loss, accuracy

In [27]:
# モデルのインスタンス、損失関数、最適化関数の設定
# モデルのインスタンス作成時にGPUに送る
model = Nlp83(len(word_id)+2, 300, 4, 50, len(word_id)+1).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# データロード時にバッチサイズを指定する(今回は4とする)
dataloader_train = DataLoader(dataset_train, batch_size=4, shuffle=True,
                        collate_fn=my_collate_fn)

# 学習の開始
# 10エポックで終了とする
for ep in range(10):
  model.train()
  for xs, ys in dataloader_train:
    # 学習時に使用するベクトルをGPUに移動する
    xs = xs.to(device)
    ys = ys.to(device)
    output = model(xs) 
    loss = criterion(output, ys) 
    optimizer.zero_grad()
    loss.backward() 
    optimizer.step()

  loss_train, accuracy_train = cal_loss_accuracy_gpu(model, criterion, dataset_train)
  loss_test, accuracy_test = cal_loss_accuracy_gpu(model, criterion, dataset_test) 
  print('エポック数', ep)
  print('[訓練データ]損失：',loss_train,'正解率:',accuracy_train)
  print('[評価データ]損失：',loss_test,'正解率:',accuracy_test)

エポック数 0
[訓練データ]損失： 1.300644040107727 正解率: 0.42185157537460327
[評価データ]損失： 1.3006950616836548 正解率: 0.4220389723777771
エポック数 1
[訓練データ]損失： 1.2570574283599854 正解率: 0.44865068793296814
[評価データ]損失： 1.2578916549682617 正解率: 0.4227886199951172
エポック数 2
[訓練データ]損失： 1.163278341293335 正解率: 0.5803973078727722
[評価データ]損失： 1.1754601001739502 正解率: 0.5682159066200256
エポック数 3
[訓練データ]損失： 1.286340594291687 正解率: 0.45239880681037903
[評価データ]損失： 1.3207647800445557 正解率: 0.4227886199951172
エポック数 4
[訓練データ]損失： 1.2519757747650146 正解率: 0.4906296730041504
[評価データ]損失： 1.278905987739563 正解率: 0.46476760506629944
エポック数 5
[訓練データ]損失： 1.165626883506775 正解率: 0.577867329120636
[評価データ]損失： 1.188205599784851 正解率: 0.5554722547531128
エポック数 6
[訓練データ]損失： 1.1831978559494019 正解率: 0.5528485774993896
[評価データ]損失： 1.2030823230743408 正解率: 0.5367316603660583
エポック数 7
[訓練データ]損失： 1.1659795045852661 正解率: 0.5776799321174622
[評価データ]損失： 1.2001930475234985 正解率: 0.54347825050354
エポック数 8
[訓練データ]損失： 1.1475555896759033 正解率: 0.596045732498169
[評価データ]損失： 1.193

In [ ]:
# 84. 単語ベクトルの導入
# 学習済み単語ベクトルのダウンロード(コピペ)
# 学習済み単語ベクトルのダウンロード
FILE_ID = "0B7XkCwpI5KDYNlNUTTlSS21pQmM"
FILE_NAME = "GoogleNews-vectors-negative300.bin.gz"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=$FILE_ID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$FILE_ID" -O $FILE_NAME && rm -rf /tmp/cookies.txt

In [31]:
from gensim.models import KeyedVectors

embd_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [32]:
# 事前学習済みの単語ベクトルで初期化するための行列を作成する
# 行列のサイズを決める(単語数、単語ベクトルの次元)
vocab_size = len(word_id)+2
embd_size = 300
# そのサイズの0行列を作成
embd_weights = np.zeros((vocab_size, embd_size))
# 単語ID辞書に登録されている単語が事前学習済みの単語ベクトルに存在するなら
# その単語ベクトルを取得する
for i, word  in enumerate(word_id.keys()):
    if word in embd_model.index2word:
        embd_weights[i] = embd_model[word]

# 配列の型が異なるので合わせておく  
embd_weights = torch.from_numpy(embd_weights.astype((np.float32)))
# 確認
print(embd_weights)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.2910,  0.1787,  0.0500,  ..., -0.0228,  0.1177,  0.3535],
        [ 0.0703,  0.0869,  0.0879,  ..., -0.0476,  0.0145, -0.0625],
        ...,
        [ 0.2207,  0.1963,  0.0649,  ...,  0.1011,  0.0388, -0.1152],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])


In [33]:
# モデルの設定(81の流用)
class Nlp84(nn.Module):
  def __init__(self, vocab_size, embd_size, output_size, hid_size, padding_idx, embd_weights):
    super(Nlp84, self).__init__()

    # 事前学習済み単語ベクトルを使う場合は
    # Embedding.from_pretrainedとし、パラメータに作成した行列を指定する
    self.embd = nn.Embedding.from_pretrained(embd_weights, padding_idx=padding_idx)
    self.rnn = nn.RNN(embd_size, hid_size, nonlinearity='tanh', batch_first=True)
    self.l1=nn.Linear(hid_size, output_size)
        
  def forward(self,x):
    ex = self.embd(x)
    out, h_T = self.rnn(ex)
    h1 = torch.softmax(self.l1(out[:,-1,:]), dim=-1)
    return h1

In [34]:
# モデルのインスタンス、損失関数、最適化関数の設定(GPUには送らない)
# 引数に作成した行列を追加する
model = Nlp84(len(word_id)+2, 300, 4, 50, len(word_id)+1, embd_weights)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# 今回もバッチサイズ4とする
dataloader_train = DataLoader(dataset_train, batch_size=4, shuffle=True,
                        collate_fn=my_collate_fn)

# 学習の開始
# 10エポックで終了とする
for ep in range(10):
  model.train()
  for xs, ys in dataloader_train:
    output = model(xs) 
    loss = criterion(output, ys) 
    optimizer.zero_grad()
    loss.backward() 
    optimizer.step()

  loss_train, accuracy_train = cal_loss_accuracy(model, criterion, dataset_train)
  loss_test, accuracy_test = cal_loss_accuracy(model, criterion, dataset_test) 
  print('エポック数', ep)
  print('[訓練データ]損失：',loss_train,'正解率:',accuracy_train)
  print('[評価データ]損失：',loss_test,'正解率:',accuracy_test)

エポック数 0
[訓練データ]損失： 1.2789360284805298 正解率: 0.43412667512893677
[評価データ]損失： 1.2973862886428833 正解率: 0.32383808493614197
エポック数 1
[訓練データ]損失： 1.2983262538909912 正解率: 0.42185157537460327
[評価データ]損失： 1.2979673147201538 正解率: 0.4220389723777771
エポック数 2
[訓練データ]損失： 1.319261908531189 正解率: 0.3957083821296692
[評価データ]損失： 1.3188124895095825 正解率: 0.3958021104335785
エポック数 3
[訓練データ]損失： 1.2765278816223145 正解率: 0.39683282375335693
[評価データ]損失： 1.2771085500717163 正解率: 0.3965517282485962
エポック数 4
[訓練データ]損失： 1.3178727626800537 正解率: 0.4196026921272278
[評価データ]損失： 1.319366455078125 正解率: 0.4227886199951172
エポック数 5
[訓練データ]損失： 1.2825437784194946 正解率: 0.4090142548084259
[評価データ]損失： 1.2890218496322632 正解率: 0.4227886199951172
エポック数 6
[訓練データ]損失： 1.289052128791809 正解率: 0.42185157537460327
[評価データ]損失： 1.2888617515563965 正解率: 0.4220389723777771
エポック数 7
[訓練データ]損失： 1.2895276546478271 正解率: 0.42185157537460327
[評価データ]損失： 1.289247751235962 正解率: 0.4220389723777771
エポック数 8
[訓練データ]損失： 1.3198267221450806 正解率: 0.3957083821296692
[評価データ]損

In [35]:
# 85. 双方向RNN・多層化

# 双方向RNN・多層化はnn.RNNのパラメータを変更するだけで実装可能
# モデルの設定(81の流用(事前学習済み単語ベクトルでは評価が下がったので使用しない))
class Nlp85(nn.Module):
  def __init__(self, vocab_size, embd_size, output_size, hid_size, padding_idx):
    super(Nlp85, self).__init__()
    self.embd = nn.Embedding(vocab_size, embd_size, padding_idx=padding_idx)

    # 双方向にするにはbidirectional=True
    # 多層化にはnum_layersの数値を変更すればよい(デフォルトは1)
    self.rnn = nn.RNN(embd_size, hid_size, nonlinearity='tanh', batch_first=True,
                      num_layers=2, bidirectional=True)
    
    # この時RNNのあとの分散表現のサイズが2倍になっているので注意
    self.l1=nn.Linear(hid_size*2, output_size)
        
  def forward(self,x):
    ex = self.embd(x)
    out, h_T = self.rnn(ex)
    h1 = torch.softmax(self.l1(out[:,-1,:]), dim=-1)
    return h1

In [36]:
# モデルのインスタンス、損失関数、最適化関数の設定
model = Nlp85(len(word_id)+2, 300, 4, 50, len(word_id)+1)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

dataloader_train = DataLoader(dataset_train, batch_size=4, shuffle=True,
                        collate_fn=my_collate_fn)

# 学習の開始
# 10エポックで終了とする
for ep in range(10):
  model.train()
  for xs, ys in dataloader_train:
    output = model(xs) 
    loss = criterion(output, ys) 
    optimizer.zero_grad()
    loss.backward() 
    optimizer.step()

  loss_train, accuracy_train = cal_loss_accuracy(model, criterion, dataset_train)
  loss_test, accuracy_test = cal_loss_accuracy(model, criterion, dataset_test) 
  print('エポック数', ep)
  print('[訓練データ]損失：',loss_train,'正解率:',accuracy_train)
  print('[評価データ]損失：',loss_test,'正解率:',accuracy_test)

エポック数 0
[訓練データ]損失： 1.2618197202682495 正解率: 0.422132670879364
[評価データ]損失： 1.2628974914550781 正解率: 0.4227886199951172
エポック数 1
[訓練データ]損失： 1.1313163042068481 正解率: 0.6144115328788757
[評価データ]損失： 1.3472105264663696 正解率: 0.3958021104335785
エポック数 2
[訓練データ]損失： 1.0827618837356567 正解率: 0.6619190573692322
[評価データ]損失： 1.093271017074585 正解率: 0.6514242887496948
エポック数 3
[訓練データ]損失： 1.1246137619018555 正解率: 0.6196589469909668
[評価データ]損失： 1.1422545909881592 正解率: 0.6019490361213684
エポック数 4
[訓練データ]損失： 1.3479722738265991 正解率: 0.3957083821296692
[評価データ]損失： 1.348515510559082 正解率: 0.3950524628162384
エポック数 5
[訓練データ]損失： 1.3478336334228516 正解率: 0.3957083821296692
[評価データ]損失： 1.3480664491653442 正解率: 0.3958021104335785
エポック数 6
[訓練データ]損失： 1.3193633556365967 正解率: 0.4242878556251526
[評価データ]損失： 1.3209311962127686 正解率: 0.4227886199951172
エポック数 7
[訓練データ]損失： 1.1567015647888184 正解率: 0.5718703269958496
[評価データ]損失： 1.1563701629638672 正解率: 0.5787106156349182
エポック数 8
[訓練データ]損失： 1.2206745147705078 正解率: 0.5223950743675232
[評価データ]損失： 1.

In [37]:
# 86. 畳み込みニューラルネットワーク (CNN)
# モデルの作成
class Nlp86(nn.Module):
  def __init__(self, vocab_size, embd_size, output_size, hid_size, padding_idx, kernel_size, stride, padding):
    super(Nlp86, self).__init__()
    self.embd = nn.Embedding(vocab_size, embd_size, padding_idx=padding_idx)

    # CNNはPyTorchのConv2dを用いる
    # kernel_size(フィルターのサイズ)、ストライド、パディングの有無(オプションは0)を決定
    # 最初の１はCNNの層の数（おそらく）
    self.cnn = nn.Conv2d(1, hid_size, (kernel_size, embd_size), stride, (padding, 0))
    # 活性化関数は今回はreluを用いる
    self.g = nn.ReLU()
    self.l1=nn.Linear(hid_size, output_size)
        
  def forward(self,x):
    # 埋め込み層に通した後、unsqueezeでバッチとして直す(CNNに通すため)
    ex = self.embd(x).unsqueeze(1)
    cn = self.cnn(ex)
    # 3個のトークンを活性化関数(今回はrelu)に入力する
    pt = self.g(cn.squeeze(3))
    # max_pool1d(一次元)を用いて最大値プーリングを行う
    # pt.size()[2]はその文の単語数を表している
    # つまり文全体を範囲としてプーリングを行う
    c = F.max_pool1d(pt, pt.size()[2])
    # プーリングで得たcの要素数は3つあるが、最後の軸が必要ないので、squeezeで省く
    h1 = torch.softmax(self.l1(c.squeeze(2)), dim=-1)
    return h1

In [38]:
# 確認
# モデルの引数はRNNとほぼ同じで、最後の3つがフィルターのサイズ(3)、ストライド(1)、パディングの有無(有)
model = Nlp86(len(word_id)+2, 300, 4, 50, len(word_id)+1, 3, 1, 1)

# 今回は確認なのでバッチ処理で行う
dataloader_train = DataLoader(dataset_train, batch_size=len(dataset_train), shuffle=True,
                        collate_fn=my_collate_fn)
for xs, ys in dataloader_train:
  y1_hat=model(xs)
  print(y1_hat)

tensor([[0.2035, 0.1997, 0.3897, 0.2071],
        [0.2001, 0.1596, 0.4919, 0.1484],
        [0.2259, 0.1484, 0.4345, 0.1913],
        ...,
        [0.1860, 0.0986, 0.4852, 0.2302],
        [0.1838, 0.1343, 0.5296, 0.1523],
        [0.2429, 0.1552, 0.4276, 0.1743]], grad_fn=<SoftmaxBackward>)


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [39]:
# 87. 確率的勾配降下法によるCNNの学習
# RNNのときと変わりはない
# モデルのインスタンス、損失関数、最適化関数の設定
model = Nlp86(len(word_id)+2, 300, 4, 50, len(word_id)+1, 3, 1, 1)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# バッチサイズは同じく4とする
dataloader_train = DataLoader(dataset_train, batch_size=4, shuffle=True,
                        collate_fn=my_collate_fn)

# 学習の開始
# 10エポックで終了とする
for ep in range(10):
  model.train()
  for xs, ys in dataloader_train:
    output = model(xs) 
    loss = criterion(output, ys) 
    optimizer.zero_grad()
    loss.backward() 
    optimizer.step()

  loss_train, accuracy_train = cal_loss_accuracy(model, criterion, dataset_train)
  loss_test, accuracy_test = cal_loss_accuracy(model, criterion, dataset_test) 
  print('エポック数', ep)
  print('[訓練データ]損失：',loss_train,'正解率:',accuracy_train)
  print('[評価データ]損失：',loss_test,'正解率:',accuracy_test)

エポック数 0
[訓練データ]損失： 0.99582839012146 正解率: 0.7579647898674011
[評価データ]損失： 1.0292918682098389 正解率: 0.7181409001350403
エポック数 1
[訓練データ]損失： 0.956272542476654 正解率: 0.7919790148735046
[評価データ]損失： 1.0095410346984863 正解率: 0.739130437374115
エポック数 2
[訓練データ]損失： 0.9396811723709106 正解率: 0.8038793206214905
[評価データ]損失： 0.9952110052108765 正解率: 0.7473763227462769
エポック数 3
[訓練データ]損失： 0.9321897625923157 正解率: 0.8094077706336975
[評価データ]損失： 0.9941712021827698 正解率: 0.7481259107589722
エポック数 4
[訓練データ]損失： 0.9273912310600281 正解率: 0.8118440508842468
[評価データ]損失： 0.9898421764373779 正解率: 0.7481259107589722
エポック数 5
[訓練データ]損失： 0.9249825477600098 正解率: 0.8130621910095215
[評価データ]損失： 0.9882882833480835 正解率: 0.7533733248710632
エポック数 6
[訓練データ]損失： 0.9230136275291443 正解率: 0.8139992356300354
[評価データ]損失： 0.9866572022438049 正解率: 0.7533733248710632
エポック数 7
[訓練データ]損失： 0.9217024445533752 正解率: 0.8145614862442017
[評価データ]損失： 0.9871091246604919 正解率: 0.7503747940063477
エポック数 8
[訓練データ]損失： 0.9208493828773499 正解率: 0.8149362802505493
[評価データ]損失： 0.9

In [ ]:
# 88. パラメータチューニング
# チューニングのためにoptunaを用いるのでインストールする
!pip install optuna

In [41]:
# 85(双方向RNN・多層化)のモデル
# 活性化関数をtanhとreluに変更できるようにする
class RNN88(nn.Module):
  def __init__(self, vocab_size, embd_size, output_size, hid_size, padding_idx, g):
    super(RNN88, self).__init__()
    self.embd = nn.Embedding(vocab_size, embd_size, padding_idx=padding_idx)
    if g == 'tanh':
      self.rnn = nn.RNN(embd_size, hid_size, nonlinearity='tanh', batch_first=True,
                        num_layers=2, bidirectional=True)
    elif g == 'relu':
      self.rnn = nn.RNN(embd_size, hid_size, nonlinearity='relu', batch_first=True,
                        num_layers=2, bidirectional=True)
    self.l1=nn.Linear(hid_size*2, output_size)
        
  def forward(self,x):
    ex = self.embd(x)
    out, h_T = self.rnn(ex)
    h1 = torch.softmax(self.l1(out[:,-1,:]), dim=-1)
    return h1

In [42]:
# 86(CNN)のモデル
class CNN88(nn.Module):
  def __init__(self, vocab_size, embd_size, output_size, hid_size, padding_idx, kernel_size, stride, padding, g):
    super(CNN88, self).__init__()
    self.embd = nn.Embedding(vocab_size, embd_size, padding_idx=padding_idx)
    self.cnn = nn.Conv2d(1, hid_size, (kernel_size, embd_size), stride, (padding, 0))
    if g == 'tanh':
      self.g = nn.Tanh()
    elif g == 'relu':
      self.g = nn.ReLU()
    self.l1=nn.Linear(hid_size, output_size)
        
  def forward(self,x):
    ex = self.embd(x).unsqueeze(1)
    cn = self.cnn(ex)
    pt = self.g(cn.squeeze(3))
    c = F.max_pool1d(pt, pt.size()[2])
    h1 = torch.softmax(self.l1(c.squeeze(2)), dim=-1)
    return h1

In [43]:
# optunaを用いてパラメータの探索を行う
import optuna
# 探索のための関数の設定
def objective(trial):
  # チューニング対象のパラメータを決める
  # 用いるモデル、埋め込み層のサイズ、隠れ層のサイズ、学習率、バッチサイズを対象とする
  model_category = trial.suggest_categorical("category", ["RNN","CNN"])
  embd_size = int(trial.suggest_discrete_uniform('embd_size', 100, 400, 100))
  hid_size = int(trial.suggest_discrete_uniform('hid_size', 50, 200, 50))
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-3, 1e-1)
  batch_size = int(trial.suggest_discrete_uniform('batch_size', 4, 16, 4))
  g = trial.suggest_categorical("g", ["tanh", "relu"])

  # 単語の総数、単語ベクトル、出力ラベル、隠れ層の次元、Paddingの数値は固定
  vocab_size=len(word_id)+2
  padding_idx=len(word_id)+1
  output_size=4

  # モデルの設定(最後に活性化関数の選択のための引数を追加しておく)
  # RNNかCNNかを選択する
  if model_category == "RNN":
    model = RNN88(vocab_size, embd_size, output_size, hid_size, padding_idx, g)
  elif model_category == "CNN":
    # kernel_size(フィルターのサイズ)、ストライド、パディング有は固定
    model = CNN88(vocab_size, embd_size, output_size, hid_size, padding_idx, 3, 1, 1, g)

  # 損失関数、最適化関数の設定
  criterion = nn.CrossEntropyLoss()
  # 学習率は上で設定したパラメータの中から選択される
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

  # データの設定においてもバッチサイズは上で設定したものが選択される
  dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True,
                          collate_fn=my_collate_fn)

  # 学習の開始
  # 10エポックで終了とする
  for ep in range(10):
    model.train()
    for xs, ys in dataloader_train:
      output = model(xs) 
      loss = criterion(output, ys) 
      optimizer.zero_grad()
      loss.backward() 
      optimizer.step()

  # 検証用のデータを用いて、ベストなパラメータを決定する
  loss_valid, accuracy_valid = cal_loss_accuracy(model, criterion, dataset_valid)

  # 評価に用いるのは検証データの損失とする(optunaが最小を選択するため)
  return loss_valid 

In [44]:
# create_study()を用いてパラメータの探索を行う
study = optuna.create_study()
# timeoutで探索を切り上げる時間を設定(今回は短めにして1000秒)
study.optimize(objective, timeout=1000)

# study.best_trialでベストパラメータと値を取得可能
trial = study.best_trial
# 結果の表示
print('Best Accuracy:', trial.value)
print('[Best Params]')
for key, value in trial.params.items():
  print(key ,':', value)

[I 2021-07-12 00:15:57,996] A new study created in memory with name: no-name-45ed2896-aedf-4f89-b0a6-198861a89589
[I 2021-07-12 00:23:43,528] Trial 0 finished with value: 1.1324578523635864 and parameters: {'category': 'RNN', 'embd_size': 400.0, 'hid_size': 50.0, 'learning_rate': 0.015484215232821086, 'batch_size': 4.0, 'g': 'tanh'}. Best is trial 0 with value: 1.1324578523635864.
[I 2021-07-12 00:25:37,643] Trial 1 finished with value: 1.0060577392578125 and parameters: {'category': 'CNN', 'embd_size': 200.0, 'hid_size': 150.0, 'learning_rate': 0.011538353895505216, 'batch_size': 12.0, 'g': 'tanh'}. Best is trial 1 with value: 1.0060577392578125.
[I 2021-07-12 00:29:05,112] Trial 2 finished with value: 0.9895407557487488 and parameters: {'category': 'CNN', 'embd_size': 400.0, 'hid_size': 150.0, 'learning_rate': 0.021567578797463297, 'batch_size': 12.0, 'g': 'relu'}. Best is trial 2 with value: 0.9895407557487488.
[I 2021-07-12 00:32:07,079] Trial 3 finished with value: 1.0736619234085

Best Accuracy: 0.9895407557487488
[Best Params]
category : CNN
embd_size : 400.0
hid_size : 150.0
learning_rate : 0.021567578797463297
batch_size : 12.0
g : relu


In [45]:
# 求めたパラメータで再度学習を行い評価データの正解率を求める
# 単語の総数、単語ベクトル、出力ラベル、隠れ層の次元、Paddingの数値は固定
vocab_size=len(word_id)+2
padding_idx=len(word_id)+1
output_size=4

# ベストパラメータを取得
# float型になっているので、sizeなどはint型に直しておく
model_category = trial.params['category']
embd_size = int(trial.params['embd_size'])
hid_size = int(trial.params['hid_size'])
learning_rate = trial.params['learning_rate']
batch_size = int(trial.params['batch_size'])
g = trial.params['g']

# モデルの設定(最後に活性化関数の選択のための引数を追加しておく)
if model_category == "RNN":
  model = RNN88(vocab_size, embd_size, output_size, hid_size, padding_idx, g)
elif model_category == "CNN":
  model = CNN88(vocab_size, embd_size, output_size, hid_size, padding_idx, 3, 1, 1, g)

# 損失関数、最適化関数の設定
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True,
                        collate_fn=my_collate_fn)

# 学習の開始
# 10エポックで終了とする
for ep in range(10):
  model.train()
  for xs, ys in dataloader_train:
    output = model(xs) 
    loss = criterion(output, ys) 
    optimizer.zero_grad()
    loss.backward() 
    optimizer.step()

# 検証用のデータを用いて、ベストなパラメータを決定する
loss_test, accuracy_test = cal_loss_accuracy(model, criterion, dataset_test)
print(accuracy_test)
# モデルの確認
print(model)

0.7646176815032959
CNN88(
  (embd): Embedding(9399, 400, padding_idx=9398)
  (cnn): Conv2d(1, 150, kernel_size=(3, 400), stride=(1, 1), padding=(1, 0))
  (g): ReLU()
  (l1): Linear(in_features=150, out_features=4, bias=True)
)


In [ ]:
# 89. 事前学習済み言語モデルからの転移学習
# BERTを用いるためにtransformersをインストールする
!pip install transformers

In [ ]:
from transformers import BertModel
from transformers import BertTokenizer
# BERTの入力となる単語id列に変換する必要がある(上で設定したidとは別)
# 変換のためにBERTを読み込む(Hugging faceに登録されている？もの)
tknz = BertTokenizer.from_pretrained('bert-base-cased')

In [48]:
# 与えられた単語列に対しBERT用の単語idを返す関数
def return_bert_id(text):
  # 問80で作成したものと同じ前処理を施す
  table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
  words = text.translate(table).split()
  # encode()で文を単語id列に変換する(80とは違い文をまとめて変換可能)
  # 辞書に登録されていない単語も自動的にidに変換される(id:100)
  ids = tknz.encode(words)
  return ids

# 確認(適当な記事見出しを選択)
text = train['TITLE'][0]
print(text)
# BERTは文頭トークン([CLS])と文末トークン([SEP])がつく(idは101と102)
print(return_bert_id(text))

Bouygues confirms improved offer for Vivendi's SFR
[101, 100, 26064, 4725, 2906, 1111, 100, 188, 100, 102]


In [49]:
# BERT用データセットを作成する(基本的には前と同じ)
class MyDataset_bert(Dataset):
  def __init__(self, xdata, ydata, return_bert_id):
    self.data = xdata
    self.label = ydata
    self.return_bert_id = return_bert_id

  def __len__(self):  
    return len(self.label)

  def __getitem__(self, idx):
    # 上で作成したBERT用の関数を用いて単語列をidに変換し返す
    text = self.data[idx]
    x = self.return_bert_id(text)
    y = self.label[idx]
    return x,y

# 同じくDataloaderの際にPaddingを行うようにする
def my_collate_bert_fn(batch):
  xdata, ydata = list(zip(*batch))
  xs = list(xdata)
  # Paddingを行うときにPaddingを行った箇所を示すマスク行列を作成する
  # マスクはPadding以外の部分が1となるような行列である
  xs1, xmsk = [], []
  for k in range(len(xs)):
    ids = xs[k]
    xs1.append(torch.LongTensor(ids))
    xmsk.append(torch.LongTensor([1]*len(ids)))
  # Paddingの数値は0とする
  xs1 = pad_sequence(xs1, batch_first=True, padding_value=0)
  xmsk = pad_sequence(xmsk, batch_first=True, padding_value=0)
  
  ys = list(ydata)
  ys1 = torch.LongTensor(ys)

  return xs1, ys1, xmsk

In [50]:
# ラベルに関しては必要ないが一応ここでも記述しておく
category_dict = {'b': 0, 't': 1, 'e':2, 'm':3}
Y_train = torch.tensor(list(map(lambda x: category_dict[x], train['CATEGORY'])))
Y_valid = torch.tensor(list(map(lambda x: category_dict[x], valid['CATEGORY'])))
Y_test = torch.tensor(list(map(lambda x: category_dict[x], test['CATEGORY'])))

# それぞれのデータセットを作成
dataset_bert_train = MyDataset_bert(train['TITLE'], Y_train, return_bert_id)
dataset_bert_valid = MyDataset_bert(valid['TITLE'], Y_valid, return_bert_id)
dataset_bert_test = MyDataset_bert(test['TITLE'], Y_test, return_bert_id)

In [51]:
# 確認
dataloader_train = DataLoader(dataset_bert_train, batch_size=4, shuffle=True,
                        collate_fn=my_collate_bert_fn)
d1 = dataloader_train.__iter__()
xs, ys, xmsk = d1.next()
print(xs)
print(xmsk)
print(len(xs))
print(ys)

tensor([[  101,   100, 21530,  6386,  1485,   125,  2370,  1822,  3075,  1104,
           100,   188,  1963,   102],
        [  101,  4754,   100,  5620,  1398,  1422,  4288,  7617,   100,  1335,
          5691,   102,     0,     0],
        [  101, 26356,   100,  2818,  6300,  1257,  1322,  1104, 13274, 18908,
           102,     0,     0,     0],
        [  101,   100,   122,  3461,   100,  5351,  1136, 20342,  2631,  5351,
          1308,   102,     0,     0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])
4
tensor([0, 2, 0, 3])


In [56]:
# モデルの実装
class Bert89(nn.Module):
  def __init__(self, bert):
    super(Bert89, self).__init__()
    # bertを読み込む
    self.bert = bert
    # bertの出力は768次元となる
    # またラベルは4次元で固定なのでここで指定しておく
    self.cls = nn.Linear(768, 4)
  
  # 計算時の引数としてマスクを追加する
  def forward(self,x1,x2):
    # attention_maskでマスクを渡せる
    bout = self.bert(input_ids=x1, attention_mask=x2)
    bs = len(bout[0])
    # bout[0][i][0]はバッチ内のi番目の文に
    # 対する[CLS]の埋め込み表現
    # これをリストで集めstackで連結する
    h0 = [ bout[0][i][0] for i in range(bs)]
    h0 = torch.stack(h0,dim=0)
    # 最終的に先頭の[CLS]の埋め込み表現を用いてラベルの予測をする
    # 今回はsoftmax関数は用いない
    return self.cls(h0)

In [57]:
# 正解率と損失の計算のための関数
def cal_loss_accuracy_bert(model, dataset):
  # BERTはサイズが大きいのでバッチ処理を行うとメモリが不足するので、1つずつ正解を見る
  dataloader = DataLoader(dataset, batch_size=1, shuffle=False,
                        collate_fn=my_collate_bert_fn)
  real_data_num, ok = 0, 0
  model.eval()
  with torch.no_grad():
    for xs, ys, xmsk in dataloader:
      output = model(xs, xmsk)
      # 正解率のみ計算する
      ans = torch.argmax(output,dim=1).item()
      if ans == ys:
        ok += 1
      real_data_num += 1

    accuracy = ok / real_data_num
  return accuracy

In [59]:
# BERTを読み込む2
bert = BertModel.from_pretrained('bert-base-cased')
# 損失関数、モデルのインスタンス、最適化関数の設定
criterion = nn.CrossEntropyLoss()
model = Bert89(bert)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# バッチサイズは4とする
dataloader_train = DataLoader(dataset_bert_train, batch_size=4, shuffle=True,
                        collate_fn=my_collate_bert_fn)

# 学習の開始(流れは同じ)
# 5エポックで終了とする(時間短縮のため)
for ep in range(5):
  model.train()
  for xs, ys, xmsk in dataloader_train:
    output = model(xs, xmsk) 
    loss = criterion(output, ys) 
    optimizer.zero_grad()
    loss.backward() 
    optimizer.step()

  accuracy_train = cal_loss_accuracy_bert(model, dataset_bert_train)
  accuracy_test = cal_loss_accuracy_bert(model, dataset_bert_test) 
  print('エポック数', ep)
  print('[訓練データ]正解率:',accuracy_train)
  print('[評価データ]正解率:',accuracy_test)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predic

エポック数 0
[訓練データ]正解率: 0.8897113943028486
[評価データ]正解率: 0.856071964017991
エポック数 1
[訓練データ]正解率: 0.9145427286356822
[評価データ]正解率: 0.8590704647676162
エポック数 2
[訓練データ]正解率: 0.9347826086956522
[評価データ]正解率: 0.8755622188905547
エポック数 3
[訓練データ]正解率: 0.9416229385307346
[評価データ]正解率: 0.8650674662668666
エポック数 4
[訓練データ]正解率: 0.9580209895052474
[評価データ]正解率: 0.8718140929535232
